<a href="https://colab.research.google.com/github/SARA3SAEED/LLM-2/blob/main/12_semantic_search_on_arabic_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semanitc Search on Arabic Wikipedia

Team:  
- **Mohammed Albrayh**
- **Mohammed Aljowaie**

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pandas as pd

In [ ]:
model = SentenceTransformer("Omartificial-Intelligence-Space/Arabic-labse")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  (3): Normalize()
)

In [ ]:
! pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
docs = load_dataset(f"Cohere/wikipedia-22-12-ar-embeddings")

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3113764 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
docs['train']

Dataset({
    features: ['id', 'title', 'text', 'url', 'wiki_id', 'views', 'paragraph_id', 'langs', 'emb'],
    num_rows: 3113764
})

In [ ]:
documents = docs['train']['text'][:1000]

In [ ]:
documents[0]

'يوتيوب هو موقع ويب يسمح لمستخدميه برفع التسجيلات المرئية مجانًا ومشاهدتها عبر البث الحي (بدل التنزيل) ومشاركتها والتعليق عليها وغير ذلك. أسسه في 14 فبراير سنة 2005م ثلاث موظفين سابقين من شركة باي بال هم تشاد هيرلي وستيف تشين وجاود كريم، في مدينة سان برونو، وكان يستخدم تقنية برنامج أدوبي فلاش لعرض المقاطع المتحركة، أما الآن فيعتمد تقنية إتش تي إم إل 5. ومحتوى الموقع يتنوع بين مقاطع الأفلام، والتلفاز، والموسيقى، وكذلك الفيديو المنتج من قبل الهواة، وغيرها. وهو حاليًا مزود بأكثر من ألفي موظف. وفي أكتوبر 2006 أعلنت شركة Google التوصل لاتفاقية لشراء الموقع مقابل 1.65 مليار دولار أمريكي، أي ما يعادل 1.31 مليار يورو. ويعتبر الموقع من مواقع ويب 2.0.'

In [ ]:
embeddings = model.encode(documents, convert_to_tensor=True).to('cpu')
embeddings.shape

torch.Size([1000, 768])

In [ ]:
def semantic_search(query,embeddings, documents):
    query_embedding = model.encode(query)

    document_embeddings = embeddings
    scores = [scipy.spatial.distance.cosine(query_embedding, doc) for doc in document_embeddings]

    most_similar_docs = sorted(zip(documents, scores), key=lambda x: x[1])[:3]
    print("Most similar document:")
    for doc, score in most_similar_docs:
        print(f"{doc}: {1 - score:.4f}")

In [ ]:
semantic_search("متى تأسس اليوتيوب",embeddings,documents)

Most similar document:
في كانون الثاني (يناير) 2009، أطلق اليوتيوب «اليوتيوب للتلفزيون»، وهو إصدار من موقع الويب مصمم خصيصًا لأجهزة فك التشفير وأجهزة الوسائط التلفزيونية الأخرى مع متصفحات الويب، مما سمح في البداية بمشاهدة مقاطع الفيديو الخاصة به على مشغل ألعاب الفيديو("بلاي ستايشن" 3 ووي).: 0.5909
يستخدم اليوتيوب بشكل أساسي تنسيقات الفيديو «أف بي 9» و«أتش 264»، و«البث الديناميكي التكيفي عبر بروتوكول أش تي تي بي» بحلول يناير 2019، بدأ اليوتيوب في طرح مقاطع الفيديو بتنسيق «أي أف 1"»".: 0.5779
في أوائل عام 2018، بدأ "ليور كوهين" في التلميح إلى احتمال إطلاق خدمة «بث الموسيقى الجديدة» للاشتراك في اليوتيوب، وهي منصة تنافس خدمات أخرى مثل سبوتيفاي وأبل ميوزك. في 22 مايو 2018، تم إطلاق منصة بث الموسيقى المسماة «اليوتيوب ميوزيك».: 0.5464
